In [ ]:
# accurate

import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
import joblib
import tkinter as tk
from tkinter import filedialog

def rgb_to_hsi(img):
    # Your rgb_to_hsi function code here

def extract_features(image):
    # Your extract_features function code here

# Load the trained K-NN model and scaler
knn_model = joblib.load('knn_model.pkl')
scaler = joblib.load('scaler.pkl')

# Function to classify an image
def classify_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Extract features
    features = extract_features(image)
    
    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])
        
        # Predict the grade
        grade = knn_model.predict(scaled_features)
        
        return grade[0]
    else:
        return "Error: Unable to extract features from the image."

# Function to open file explorer for picking an image
def open_file_explorer():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    # Open file explorer and return the selected file path
    file_path = filedialog.askopenfilename()

    if file_path:
        # If a file is selected, classify the image
        predicted_grade = classify_image(file_path)
        print("Predicted Grade:", predicted_grade)
    else:
        print("No file selected.")

# Call the function to open file explorer
open_file_explorer()


In [8]:
#for paper

#zoom gA

import cv2
import os
import numpy as np


# Function to crop zoom the image within the bounding box and resize to a uniform size
def crop_zoom(image, x, y, w, h, zoom_factor=1.2, output_size=(800, 800)):
    # Calculate the zoomed area
    new_w = int(w / zoom_factor)
    new_h = int(h / zoom_factor)
    
    # Ensure the coordinates are within image boundaries
    start_x = max(x + (w - new_w) // 2, 0)
    start_y = max(y + (h - new_h) // 2, 0)
    end_x = min(start_x + new_w, image.shape[1])
    end_y = min(start_y + new_h, image.shape[0])
    
    # Crop the image to the zoomed area
    cropped_image = image[start_y:end_y, start_x:end_x]

    output_file_path = os.path.join(target_directory, "cropped.jpg")
    cv2.imwrite(output_file_path, cropped_image)
    
    # Resize the cropped image to the desired output size
    resized_image = cv2.resize(cropped_image, output_size)
    
    return resized_image

# Function to detect objects and get bounding box
def detect_and_draw_boxes(frame):
    # Convert the image to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    output_file_path = os.path.join(target_directory, "gray.jpg")
    cv2.imwrite(output_file_path, gray)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (21, 21), 0)

    output_file_path = os.path.join(target_directory, "blurred.jpg")
    cv2.imwrite(output_file_path, blurred)
    
    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    output_file_path = os.path.join(target_directory, "thresholded.jpg")
    cv2.imwrite(output_file_path, thresholded)
    
    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # Find the contour with the largest area
        main_contour = max(contours, key=cv2.contourArea)

        # output_file_path = os.path.join(target_directory, "contour.jpg")
        # cv2.imwrite(output_file_path, main_contour)
        
        # Get bounding box coordinates around the main contour
        x, y, w, h = cv2.boundingRect(main_contour)
        return x, y, w, h, frame
    else:
        return None, None, None, None, frame

# Source directory containing the images
source_directory = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\4ppr"

# Target directory for cropped zoom images
target_directory = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\4ppr output"

# Zoom factor and output size
zoom_factor = 1.2
output_size = (800, 800)

# Create the target directory if it doesn't exist
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

# Iterate over images in the source directory
for filename in os.listdir(source_directory):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        # Load the image
        input_file_path = os.path.join(source_directory, filename)
        image = cv2.imread(input_file_path)
        
        # Detect objects and get bounding box coordinates
        x, y, w, h, detected_image = detect_and_draw_boxes(image.copy())
        
        if x is not None and y is not None and w is not None and h is not None:
            # Zoom in within the bounding box and resize to uniform size
            zoomed_resized_image = crop_zoom(detected_image, x, y, w, h, zoom_factor, output_size)
            zoomed_output_file_path = os.path.join(target_directory, "grade_A_" + os.path.splitext(filename)[0] + "_zoom.jpg")
            cv2.imwrite(zoomed_output_file_path, zoomed_resized_image)

print("All cropped zoom images within bounding boxes saved to:", target_directory)


All cropped zoom images within bounding boxes saved to: C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\4ppr output


In [14]:
#working resize turn bg tro black
import cv2
import os
import numpy as np

# Function to load, resize, and remove background from an image
def load_resize_remove_background(file_path, target_size):
    # Load the image
    image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
    if image is None:
        return None
    
    # Resize the image to target size
    resized_image = cv2.resize(image, (target_size, target_size))
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask to store the foreground pixels
    mask = np.zeros_like(gray)

    # Draw contours on the mask
    cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)

    # Bitwise AND operation to extract foreground
    foreground = cv2.bitwise_and(resized_image, resized_image, mask=mask)

    # Set the background to black
    background_mask = (mask == 0)
    foreground[background_mask] = [0, 0, 0]
    
    return foreground

# Source directory containing the images
source_directory = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\4ppr output\op1"  # Replace with the actual path to your source image directory

# Target directory for resized images
target_directory = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\4ppr output\op2"  # Replace with the desired output directory for resized images

# Target size for resizing (square)
target_size = 300

# Create the target directory if it doesn't exist
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

# Iterate over images in the source directory
for filename in os.listdir(source_directory):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):  # Look for JPEG files
        # Load, resize, and remove background from the image
        input_file_path = os.path.join(source_directory, filename)
        processed_image = load_resize_remove_background(input_file_path, target_size)
        
        if processed_image is not None:
            # Save the original size processed image
            original_output_file_path = os.path.join(target_directory, os.path.splitext(filename)[0] + "_processed.png")
            cv2.imwrite(original_output_file_path, processed_image)

print("All images processed and saved to:", target_directory)


All images processed and saved to: C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\4ppr output\op2


In [2]:
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
import joblib
import tkinter as tk
from tkinter import filedialog
import math
import os
import time
import psutil

def rgb_to_hsi(img):
    with np.errstate(divide='ignore', invalid='ignore'):
        # Load image with 32 bit floats as variable type
        bgr = np.float32(img) / 255

        # Separate color channels
        blue = bgr[:, :, 0]
        green = bgr[:, :, 1]
        red = bgr[:, :, 2]

        # Calculate Intensity
        def calc_intensity(red, blue, green):
            return np.divide(blue + green + red, 3)

        # Calculate Saturation
        def calc_saturation(red, blue, green):
            minimum = np.minimum(np.minimum(red, green), blue)
            saturation = 1 - (3 / (red + green + blue + 0.001) * minimum)

            return saturation

        # Calculate Hue
        def calc_hue(red, blue, green):
            hue = np.copy(red)

            for i in range(0, blue.shape[0]):
                for j in range(0, blue.shape[1]):
                    hue[i][j] = 0.5 * ((red[i][j] - green[i][j]) + (red[i][j] - blue[i][j])) / \
                                math.sqrt((red[i][j] - green[i][j])**2 +
                                          ((red[i][j] - blue[i][j]) * (green[i][j] - blue[i][j])))
                    hue[i][j] = math.acos(hue[i][j])

                    if blue[i][j] <= green[i][j]:
                        hue[i][j] = hue[i][j]
                    else:
                        hue[i][j] = ((360 * math.pi) / 180.0) - hue[i][j]

            return hue

        # Merge channels into picture and return image
        hsi = cv2.merge((calc_hue(red, blue, green), calc_saturation(red, blue, green), calc_intensity(red, blue, green)))
        return hsi

def extract_features(image):
    # if zero dont input
    if np.any(image != [0, 0, 0]):
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Normalize RGB values to [0, 1] for accurate HSV conversion
        rgb_normalized = rgb.astype(np.float32) / 255.0

        # Convert RGB to HSV
        hsv = cv2.cvtColor(rgb_normalized, cv2.COLOR_RGB2HSV)

        # Convert to grayscale for GLCM
        gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)

        # Convert RGB to HSI
        hsi = rgb_to_hsi(image)

        # RGB
        r, g, b = np.mean(rgb[:, :, 0]), np.mean(rgb[:, :, 1]), np.mean(rgb[:, :, 2])

        # HSV
        h_hsv, s_hsv, v_hsv = np.mean(hsv[:, :, 0]), np.mean(hsv[:, :, 1]), np.mean(hsv[:, :, 2])

        # HSI (HLS approximation)
        h_hsi, s_hsi, i_hsi = np.mean(hsi[:, :, 0]), np.mean(hsi[:, :, 1]), np.mean(hsi[:, :, 2])

        # GLCM
        glcm = graycomatrix(gray, distances=[1], angles=[0, np.pi / 4, np.pi / 2, 3 * np.pi / 4],
                            symmetric=True, normed=True)
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        correlation = graycoprops(glcm, 'correlation')[0, 0]
        energy = graycoprops(glcm, 'energy')[0, 0]
        homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]

        return r, g, b, h_hsv, s_hsv, v_hsv, h_hsi, s_hsi, i_hsi, contrast, correlation, energy, homogeneity
    else:
        return None, None, None, None

# Load the trained K-NN model and scaler
knn_model = joblib.load('knn_model.pkl')
scaler = joblib.load('scaler.pkl')

# Create background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Function to classify an image
def classify_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(image)
    fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (31, 31), 0)

    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the contour with the largest area
        main_contour = max(contours, key=cv2.contourArea)

        # Draw bounding box around the main contour
        x, y, w, h = cv2.boundingRect(main_contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Zoom in to a specific area inside the bounding box
        zoom_factor = 3  # Adjust the zoom factor as needed
        center_x = x + w // 2
        center_y = y + h // 2
        zoom_size = min(w, h) // zoom_factor
        x_zoom = max(0, center_x - zoom_size // 2)
        y_zoom = max(0, center_y - zoom_size // 2)
        zoomed_image = image[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()

        # Resize the zoomed image to 100x100 pixels
        zoomed_resized = cv2.resize(zoomed_image, (32, 32))

        # Extract features from the zoomed-in frame with foreground mask
        zoomed_fg_mask = fg_mask[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()
        features = extract_features(cv2.bitwise_and(zoomed_image, zoomed_fg_mask))

    # Extract features
    features = extract_features(zoomed_resized)

    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])

        # Predict the grade
        grade = knn_model.predict(scaled_features)

        image = cv2.resize(image, (700, 700))
        # Draw the outline of the text on the zoomed image
        outline_thickness = 2
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)

        # Draw the text on the zoomed image
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the frame with bounding box
        cv2.imshow('Frame with Bounding Box', image)

        zoomed_resized = cv2.resize(zoomed_resized, (700, 700))
        # Draw the outline of the text on the zoomed image
        outline_thickness = 2
        cv2.putText(zoomed_resized, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)

        # Draw the text on the zoomed image
        cv2.putText(zoomed_resized, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the zoomed image with predicted grade
        cv2.imshow('Zoomed In with Predicted Grade', zoomed_resized)

        # Wait for a key press to close the windows
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        return grade[0]
    else:
        return "Error: Unable to extract features from the image."

# Function to classify an image and save the classified image
def classify_and_save_image(image_path, output_folder):
    start_time = time.time()
    process = psutil.Process(os.getpid())
    start_cpu = process.cpu_percent(interval=None)
    start_memory = process.memory_info().rss

    # Load the image
    image = cv2.imread(image_path)

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(image)
    fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (31, 31), 0)

    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the contour with the largest area
        main_contour = max(contours, key=cv2.contourArea)

        # Draw bounding box around the main contour
        x, y, w, h = cv2.boundingRect(main_contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Zoom in to a specific area inside the bounding box
        zoom_factor = 3  # Adjust the zoom factor as needed
        center_x = x + w // 2
        center_y = y + h // 2
        zoom_size = min(w, h) // zoom_factor
        x_zoom = max(0, center_x - zoom_size // 2)
        y_zoom = max(0, center_y - zoom_size // 2)
        zoomed_image = image[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()

        # Resize the zoomed image to 100x100 pixels
        zoomed_resized = cv2.resize(zoomed_image, (32, 32))

        # Extract features from the zoomed-in frame with foreground mask
        zoomed_fg_mask = fg_mask[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()
        features = extract_features(cv2.bitwise_and(zoomed_image, zoomed_fg_mask))

    # Extract features
    features = extract_features(zoomed_resized)

    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])

        # Predict the grade
        grade = knn_model.predict(scaled_features)

        image = cv2.resize(image, (800, 800))
        # Draw the predicted grade on the image
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Save the classified image to the output folder
        filename = os.path.basename(image_path)
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, image)

        end_time = time.time()
        end_cpu = process.cpu_percent(interval=None)
        end_memory = process.memory_info().rss

        processing_time = end_time - start_time
        cpu_usage = end_cpu - start_cpu
        ram_usage = end_memory - start_memory

        return grade[0], processing_time, cpu_usage, ram_usage
    else:
        return "Error: Unable to extract features from the image.", 0, 0, 0

# Function to classify images in a folder and save the classified images to another folder
def classify_images_in_folder(input_folder, output_folder):
    processing_times = []
    cpu_usages = []
    ram_usages = []

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Construct the full path of the input image
            image_path = os.path.join(input_folder, filename)

            # Classify the image and save the classified image to the output folder
            predicted_grade, processing_time, cpu_usage, ram_usage = classify_and_save_image(image_path, output_folder)
            processing_times.append(processing_time)
            cpu_usages.append(cpu_usage)
            ram_usages.append(ram_usage)
            print("Image:", filename, "- Predicted Grade:", predicted_grade)
    
    print("Processing Time - Min:", np.min(processing_times), "Max:", np.max(processing_times), "Mean:", np.mean(processing_times))
    print("CPU Usage - Min:", np.min(cpu_usages), "Max:", np.max(cpu_usages), "Mean:", np.mean(cpu_usages))
    print("RAM Usage - Min:", np.min(ram_usages), "Max:", np.max(ram_usages), "Mean:", np.mean(ram_usages))

# Call the function to classify images in the input folder and save the classified images to the output folder
input_folder = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\TUNA GRADE B\validation"
output_folder = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\TUNA GRADE B\val output2"
classify_images_in_folder(input_folder, output_folder)
print("B done")


Image: IMG20220515184251.jpg - Predicted Grade: B
Image: IMG20220515184251_01.jpg - Predicted Grade: B
Image: IMG20220515184254.jpg - Predicted Grade: B
Image: IMG20220515184255.jpg - Predicted Grade: B
Image: IMG20220515184256.jpg - Predicted Grade: B
Image: IMG20220515184258.jpg - Predicted Grade: B
Image: IMG20220515184304.jpg - Predicted Grade: B
Image: IMG20220515184306.jpg - Predicted Grade: B
Image: IMG20220515184307.jpg - Predicted Grade: B
Image: IMG20220515184308.jpg - Predicted Grade: B
Image: IMG20220515184309.jpg - Predicted Grade: B
Image: IMG20220515184310.jpg - Predicted Grade: B
Image: IMG20220515184311.jpg - Predicted Grade: B
Image: IMG20220515184313.jpg - Predicted Grade: B
Image: IMG20220515184314.jpg - Predicted Grade: B
Image: IMG20220515184316.jpg - Predicted Grade: B
Image: IMG20220515184318.jpg - Predicted Grade: B
Image: IMG20220515184320.jpg - Predicted Grade: B
Image: IMG20220515184322.jpg - Predicted Grade: B
Image: IMG20220515184323.jpg - Predicted Grade:

In [3]:
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
import joblib
import tkinter as tk
from tkinter import filedialog
import math
import os
import time
import psutil

def rgb_to_hsi(img):
    with np.errstate(divide='ignore', invalid='ignore'):
        # Load image with 32 bit floats as variable type
        bgr = np.float32(img) / 255

        # Separate color channels
        blue = bgr[:, :, 0]
        green = bgr[:, :, 1]
        red = bgr[:, :, 2]

        # Calculate Intensity
        def calc_intensity(red, blue, green):
            return np.divide(blue + green + red, 3)

        # Calculate Saturation
        def calc_saturation(red, blue, green):
            minimum = np.minimum(np.minimum(red, green), blue)
            saturation = 1 - (3 / (red + green + blue + 0.001) * minimum)

            return saturation

        # Calculate Hue
        def calc_hue(red, blue, green):
            hue = np.copy(red)

            for i in range(0, blue.shape[0]):
                for j in range(0, blue.shape[1]):
                    hue[i][j] = 0.5 * ((red[i][j] - green[i][j]) + (red[i][j] - blue[i][j])) / \
                                math.sqrt((red[i][j] - green[i][j])**2 +
                                          ((red[i][j] - blue[i][j]) * (green[i][j] - blue[i][j])))
                    hue[i][j] = math.acos(hue[i][j])

                    if blue[i][j] <= green[i][j]:
                        hue[i][j] = hue[i][j]
                    else:
                        hue[i][j] = ((360 * math.pi) / 180.0) - hue[i][j]

            return hue

        # Merge channels into picture and return image
        hsi = cv2.merge((calc_hue(red, blue, green), calc_saturation(red, blue, green), calc_intensity(red, blue, green)))
        return hsi

def extract_features(image):
    # if zero dont input
    if np.any(image != [0, 0, 0]):
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Normalize RGB values to [0, 1] for accurate HSV conversion
        rgb_normalized = rgb.astype(np.float32) / 255.0

        # Convert RGB to HSV
        hsv = cv2.cvtColor(rgb_normalized, cv2.COLOR_RGB2HSV)

        # Convert to grayscale for GLCM
        gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)

        # Convert RGB to HSI
        hsi = rgb_to_hsi(image)

        # RGB
        r, g, b = np.mean(rgb[:, :, 0]), np.mean(rgb[:, :, 1]), np.mean(rgb[:, :, 2])

        # HSV
        h_hsv, s_hsv, v_hsv = np.mean(hsv[:, :, 0]), np.mean(hsv[:, :, 1]), np.mean(hsv[:, :, 2])

        # HSI (HLS approximation)
        h_hsi, s_hsi, i_hsi = np.mean(hsi[:, :, 0]), np.mean(hsi[:, :, 1]), np.mean(hsi[:, :, 2])

        # GLCM
        glcm = graycomatrix(gray, distances=[1], angles=[0, np.pi / 4, np.pi / 2, 3 * np.pi / 4],
                            symmetric=True, normed=True)
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        correlation = graycoprops(glcm, 'correlation')[0, 0]
        energy = graycoprops(glcm, 'energy')[0, 0]
        homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]

        return r, g, b, h_hsv, s_hsv, v_hsv, h_hsi, s_hsi, i_hsi, contrast, correlation, energy, homogeneity
    else:
        return None, None, None, None

# Load the trained K-NN model and scaler
knn_model = joblib.load('knn_model.pkl')
scaler = joblib.load('scaler.pkl')

# Create background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Function to classify an image
def classify_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(image)
    fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (31, 31), 0)

    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the contour with the largest area
        main_contour = max(contours, key=cv2.contourArea)

        # Draw bounding box around the main contour
        x, y, w, h = cv2.boundingRect(main_contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Zoom in to a specific area inside the bounding box
        zoom_factor = 3  # Adjust the zoom factor as needed
        center_x = x + w // 2
        center_y = y + h // 2
        zoom_size = min(w, h) // zoom_factor
        x_zoom = max(0, center_x - zoom_size // 2)
        y_zoom = max(0, center_y - zoom_size // 2)
        zoomed_image = image[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()

        # Resize the zoomed image to 100x100 pixels
        zoomed_resized = cv2.resize(zoomed_image, (32, 32))

        # Extract features from the zoomed-in frame with foreground mask
        zoomed_fg_mask = fg_mask[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()
        features = extract_features(cv2.bitwise_and(zoomed_image, zoomed_fg_mask))

    # Extract features
    features = extract_features(zoomed_resized)

    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])

        # Predict the grade
        grade = knn_model.predict(scaled_features)

        image = cv2.resize(image, (700, 700))
        # Draw the outline of the text on the zoomed image
        outline_thickness = 2
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)

        # Draw the text on the zoomed image
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the frame with bounding box
        cv2.imshow('Frame with Bounding Box', image)

        zoomed_resized = cv2.resize(zoomed_resized, (700, 700))
        # Draw the outline of the text on the zoomed image
        outline_thickness = 2
        cv2.putText(zoomed_resized, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)

        # Draw the text on the zoomed image
        cv2.putText(zoomed_resized, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the zoomed image with predicted grade
        cv2.imshow('Zoomed In with Predicted Grade', zoomed_resized)

        # Wait for a key press to close the windows
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        return grade[0]
    else:
        return "Error: Unable to extract features from the image."

# Function to classify an image and save the classified image
def classify_and_save_image(image_path, output_folder):
    start_time = time.time()
    process = psutil.Process(os.getpid())
    start_cpu = process.cpu_percent(interval=None)
    start_memory = process.memory_info().rss

    # Load the image
    image = cv2.imread(image_path)

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(image)
    fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (31, 31), 0)

    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the contour with the largest area
        main_contour = max(contours, key=cv2.contourArea)

        # Draw bounding box around the main contour
        x, y, w, h = cv2.boundingRect(main_contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Zoom in to a specific area inside the bounding box
        zoom_factor = 3  # Adjust the zoom factor as needed
        center_x = x + w // 2
        center_y = y + h // 2
        zoom_size = min(w, h) // zoom_factor
        x_zoom = max(0, center_x - zoom_size // 2)
        y_zoom = max(0, center_y - zoom_size // 2)
        zoomed_image = image[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()

        # Resize the zoomed image to 100x100 pixels
        zoomed_resized = cv2.resize(zoomed_image, (32, 32))

        # Extract features from the zoomed-in frame with foreground mask
        zoomed_fg_mask = fg_mask[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()
        features = extract_features(cv2.bitwise_and(zoomed_image, zoomed_fg_mask))

    # Extract features
    features = extract_features(zoomed_resized)

    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])

        # Predict the grade
        grade = knn_model.predict(scaled_features)

        image = cv2.resize(image, (800, 800))
        # Draw the predicted grade on the image
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Save the classified image to the output folder
        filename = os.path.basename(image_path)
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, image)

        end_time = time.time()
        end_cpu = process.cpu_percent(interval=None)
        end_memory = process.memory_info().rss

        processing_time = end_time - start_time
        cpu_usage = (end_cpu - start_cpu) / psutil.cpu_count()  # Convert to percent
        ram_usage = (end_memory - start_memory) / (1024 * 1024)  # Convert to MB

        return grade[0], processing_time, cpu_usage, ram_usage
    else:
        return "Error: Unable to extract features from the image.", 0, 0, 0

# Function to classify images in a folder and save the classified images to another folder
def classify_images_in_folder(input_folder, output_folder):
    processing_times = []
    cpu_usages = []
    ram_usages = []

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Construct the full path of the input image
            image_path = os.path.join(input_folder, filename)

            # Classify the image and save the classified image to the output folder
            predicted_grade, processing_time, cpu_usage, ram_usage = classify_and_save_image(image_path, output_folder)
            processing_times.append(processing_time)
            cpu_usages.append(cpu_usage)
            ram_usages.append(ram_usage)
            print("Image:", filename, "- Predicted Grade:", predicted_grade)
    
    print("Processing Time - Min:", np.min(processing_times), "Max:", np.max(processing_times), "Mean:", np.mean(processing_times))
    print("CPU Usage (%) - Min:", np.min(cpu_usages), "Max:", np.max(cpu_usages), "Mean:", np.mean(cpu_usages))
    print("RAM Usage (MB) - Min:", np.min(ram_usages), "Max:", np.max(ram_usages), "Mean:", np.mean(ram_usages))

# Call the function to classify images in the input folder and save the classified images to the output folder
input_folder = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\TUNA GRADE B\validation"
output_folder = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\TUNA GRADE B\val output2"
classify_images_in_folder(input_folder, output_folder)
print("B done")


Image: IMG20220515184251.jpg - Predicted Grade: B
Image: IMG20220515184251_01.jpg - Predicted Grade: B
Image: IMG20220515184254.jpg - Predicted Grade: B
Image: IMG20220515184255.jpg - Predicted Grade: B
Image: IMG20220515184256.jpg - Predicted Grade: B
Image: IMG20220515184258.jpg - Predicted Grade: B
Image: IMG20220515184304.jpg - Predicted Grade: B
Image: IMG20220515184306.jpg - Predicted Grade: B
Image: IMG20220515184307.jpg - Predicted Grade: B
Image: IMG20220515184308.jpg - Predicted Grade: B
Image: IMG20220515184309.jpg - Predicted Grade: B
Image: IMG20220515184310.jpg - Predicted Grade: B
Image: IMG20220515184311.jpg - Predicted Grade: B
Image: IMG20220515184313.jpg - Predicted Grade: B
Image: IMG20220515184314.jpg - Predicted Grade: B
Image: IMG20220515184316.jpg - Predicted Grade: B
Image: IMG20220515184318.jpg - Predicted Grade: B
Image: IMG20220515184320.jpg - Predicted Grade: B
Image: IMG20220515184322.jpg - Predicted Grade: B
Image: IMG20220515184323.jpg - Predicted Grade:

In [4]:
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
import joblib
import tkinter as tk
from tkinter import filedialog
import math
import os
import time
import psutil

def rgb_to_hsi(img):
    with np.errstate(divide='ignore', invalid='ignore'):
        # Load image with 32 bit floats as variable type
        bgr = np.float32(img) / 255

        # Separate color channels
        blue = bgr[:, :, 0]
        green = bgr[:, :, 1]
        red = bgr[:, :, 2]

        # Calculate Intensity
        def calc_intensity(red, blue, green):
            return np.divide(blue + green + red, 3)

        # Calculate Saturation
        def calc_saturation(red, blue, green):
            minimum = np.minimum(np.minimum(red, green), blue)
            saturation = 1 - (3 / (red + green + blue + 0.001) * minimum)

            return saturation

        # Calculate Hue
        def calc_hue(red, blue, green):
            hue = np.copy(red)

            for i in range(0, blue.shape[0]):
                for j in range(0, blue.shape[1]):
                    hue[i][j] = 0.5 * ((red[i][j] - green[i][j]) + (red[i][j] - blue[i][j])) / \
                                math.sqrt((red[i][j] - green[i][j])**2 +
                                          ((red[i][j] - blue[i][j]) * (green[i][j] - blue[i][j])))
                    hue[i][j] = math.acos(hue[i][j])

                    if blue[i][j] <= green[i][j]:
                        hue[i][j] = hue[i][j]
                    else:
                        hue[i][j] = ((360 * math.pi) / 180.0) - hue[i][j]

            return hue

        # Merge channels into picture and return image
        hsi = cv2.merge((calc_hue(red, blue, green), calc_saturation(red, blue, green), calc_intensity(red, blue, green)))
        return hsi

def extract_features(image):
    # if zero dont input
    if np.any(image != [0, 0, 0]):
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Normalize RGB values to [0, 1] for accurate HSV conversion
        rgb_normalized = rgb.astype(np.float32) / 255.0

        # Convert RGB to HSV
        hsv = cv2.cvtColor(rgb_normalized, cv2.COLOR_RGB2HSV)

        # Convert to grayscale for GLCM
        gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)

        # Convert RGB to HSI
        hsi = rgb_to_hsi(image)

        # RGB
        r, g, b = np.mean(rgb[:, :, 0]), np.mean(rgb[:, :, 1]), np.mean(rgb[:, :, 2])

        # HSV
        h_hsv, s_hsv, v_hsv = np.mean(hsv[:, :, 0]), np.mean(hsv[:, :, 1]), np.mean(hsv[:, :, 2])

        # HSI (HLS approximation)
        h_hsi, s_hsi, i_hsi = np.mean(hsi[:, :, 0]), np.mean(hsi[:, :, 1]), np.mean(hsi[:, :, 2])

        # GLCM
        glcm = graycomatrix(gray, distances=[1], angles=[0, np.pi / 4, np.pi / 2, 3 * np.pi / 4],
                            symmetric=True, normed=True)
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        correlation = graycoprops(glcm, 'correlation')[0, 0]
        energy = graycoprops(glcm, 'energy')[0, 0]
        homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]

        return r, g, b, h_hsv, s_hsv, v_hsv, h_hsi, s_hsi, i_hsi, contrast, correlation, energy, homogeneity
    else:
        return None, None, None, None

# Load the trained K-NN model and scaler
knn_model = joblib.load('knn_model.pkl')
scaler = joblib.load('scaler.pkl')

# Create background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Function to classify an image
def classify_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(image)
    fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (31, 31), 0)

    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the contour with the largest area
        main_contour = max(contours, key=cv2.contourArea)

        # Draw bounding box around the main contour
        x, y, w, h = cv2.boundingRect(main_contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Zoom in to a specific area inside the bounding box
        zoom_factor = 3  # Adjust the zoom factor as needed
        center_x = x + w // 2
        center_y = y + h // 2
        zoom_size = min(w, h) // zoom_factor
        x_zoom = max(0, center_x - zoom_size // 2)
        y_zoom = max(0, center_y - zoom_size // 2)
        zoomed_image = image[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()

        # Resize the zoomed image to 100x100 pixels
        zoomed_resized = cv2.resize(zoomed_image, (32, 32))

        # Extract features from the zoomed-in frame with foreground mask
        zoomed_fg_mask = fg_mask[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()
        features = extract_features(cv2.bitwise_and(zoomed_image, zoomed_fg_mask))

    # Extract features
    features = extract_features(zoomed_resized)

    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])

        # Predict the grade
        grade = knn_model.predict(scaled_features)

        image = cv2.resize(image, (700, 700))
        # Draw the outline of the text on the zoomed image
        outline_thickness = 2
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)

        # Draw the text on the zoomed image
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the frame with bounding box
        cv2.imshow('Frame with Bounding Box', image)

        zoomed_resized = cv2.resize(zoomed_resized, (700, 700))
        # Draw the outline of the text on the zoomed image
        outline_thickness = 2
        cv2.putText(zoomed_resized, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)

        # Draw the text on the zoomed image
        cv2.putText(zoomed_resized, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the zoomed image with predicted grade
        cv2.imshow('Zoomed In with Predicted Grade', zoomed_resized)

        # Wait for a key press to close the windows
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        return grade[0]
    else:
        return "Error: Unable to extract features from the image."

# Function to classify an image and save the classified image
def classify_and_save_image(image_path, output_folder):
    start_time = time.time()
    process = psutil.Process(os.getpid())
    start_cpu = process.cpu_percent(interval=None)
    start_memory = process.memory_info().rss

    # Load the image
    image = cv2.imread(image_path)

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(image)
    fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (31, 31), 0)

    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the contour with the largest area
        main_contour = max(contours, key=cv2.contourArea)

        # Draw bounding box around the main contour
        x, y, w, h = cv2.boundingRect(main_contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Zoom in to a specific area inside the bounding box
        zoom_factor = 3  # Adjust the zoom factor as needed
        center_x = x + w // 2
        center_y = y + h // 2
        zoom_size = min(w, h) // zoom_factor
        x_zoom = max(0, center_x - zoom_size // 2)
        y_zoom = max(0, center_y - zoom_size // 2)
        zoomed_image = image[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()

        # Resize the zoomed image to 100x100 pixels
        zoomed_resized = cv2.resize(zoomed_image, (32, 32))

        # Extract features from the zoomed-in frame with foreground mask
        zoomed_fg_mask = fg_mask[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()
        features = extract_features(cv2.bitwise_and(zoomed_image, zoomed_fg_mask))

    # Extract features
    features = extract_features(zoomed_resized)

    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])

        # Predict the grade
        grade = knn_model.predict(scaled_features)

        image = cv2.resize(image, (800, 800))
        # Draw the predicted grade on the image
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Save the classified image to the output folder
        filename = os.path.basename(image_path)
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, image)

        end_time = time.time()
        end_cpu = process.cpu_percent(interval=None)
        end_memory = process.memory_info().rss

        processing_time = end_time - start_time
        cpu_usage = (end_cpu - start_cpu) / psutil.cpu_count()  # Convert to percent
        ram_usage = (end_memory - start_memory) / (1024 * 1024)  # Convert to MB

        return grade[0], processing_time, cpu_usage, ram_usage
    else:
        return "Error: Unable to extract features from the image.", 0, 0, 0

# Function to classify images in a folder and save the classified images to another folder
def classify_images_in_folder(input_folder, output_folder):
    processing_times = []
    cpu_usages = []
    ram_usages = []

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Construct the full path of the input image
            image_path = os.path.join(input_folder, filename)

            # Classify the image and save the classified image to the output folder
            predicted_grade, processing_time, cpu_usage, ram_usage = classify_and_save_image(image_path, output_folder)
            processing_times.append(processing_time)
            cpu_usages.append(cpu_usage)
            ram_usages.append(ram_usage)
            print("Image:", filename, "- Predicted Grade:", predicted_grade)
    
    print("Processing Time - Min:", np.min(processing_times), "Max:", np.max(processing_times), "Mean:", np.mean(processing_times))
    print("CPU Usage (%) - Min:", np.min(cpu_usages), "Max:", np.max(cpu_usages), "Mean:", np.mean(cpu_usages))
    print("RAM Usage (MB) - Min:", np.min(ram_usages), "Max:", np.max(ram_usages), "Mean:", np.mean(ram_usages))

# Call the function to classify images in the input folder and save the classified images to the output folder
input_folder = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\TUNA GRADE A\validation"
output_folder = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\TUNA GRADE A\val output2"
classify_images_in_folder(input_folder, output_folder)
print("B done")


Image: IMG20220516121036.jpg - Predicted Grade: A
Image: IMG20220516121039.jpg - Predicted Grade: A
Image: IMG20220516121040.jpg - Predicted Grade: A
Image: IMG20220516121042.jpg - Predicted Grade: A
Image: IMG20220516121043.jpg - Predicted Grade: A
Image: IMG20220516121044.jpg - Predicted Grade: A
Image: IMG20220516121045.jpg - Predicted Grade: A
Image: IMG20220516121048.jpg - Predicted Grade: A
Image: IMG20220516121051.jpg - Predicted Grade: A
Image: IMG20220516121054.jpg - Predicted Grade: A
Image: IMG20220516121055.jpg - Predicted Grade: A
Image: IMG20220516121057.jpg - Predicted Grade: A
Image: IMG20220516121058.jpg - Predicted Grade: A
Image: IMG20220516121059.jpg - Predicted Grade: A
Image: IMG20220516121100.jpg - Predicted Grade: A
Image: IMG20220516121102.jpg - Predicted Grade: A
Image: IMG20220516121104.jpg - Predicted Grade: A
Image: IMG20220516121105.jpg - Predicted Grade: A
Image: IMG20220516121106.jpg - Predicted Grade: A
Image: IMG20220516121108.jpg - Predicted Grade: A


In [5]:
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
import joblib
import tkinter as tk
from tkinter import filedialog
import math
import os
import time
import psutil

def rgb_to_hsi(img):
    with np.errstate(divide='ignore', invalid='ignore'):
        # Load image with 32 bit floats as variable type
        bgr = np.float32(img) / 255

        # Separate color channels
        blue = bgr[:, :, 0]
        green = bgr[:, :, 1]
        red = bgr[:, :, 2]

        # Calculate Intensity
        def calc_intensity(red, blue, green):
            return np.divide(blue + green + red, 3)

        # Calculate Saturation
        def calc_saturation(red, blue, green):
            minimum = np.minimum(np.minimum(red, green), blue)
            saturation = 1 - (3 / (red + green + blue + 0.001) * minimum)

            return saturation

        # Calculate Hue
        def calc_hue(red, blue, green):
            hue = np.copy(red)

            for i in range(0, blue.shape[0]):
                for j in range(0, blue.shape[1]):
                    hue[i][j] = 0.5 * ((red[i][j] - green[i][j]) + (red[i][j] - blue[i][j])) / \
                                math.sqrt((red[i][j] - green[i][j])**2 +
                                          ((red[i][j] - blue[i][j]) * (green[i][j] - blue[i][j])))
                    hue[i][j] = math.acos(hue[i][j])

                    if blue[i][j] <= green[i][j]:
                        hue[i][j] = hue[i][j]
                    else:
                        hue[i][j] = ((360 * math.pi) / 180.0) - hue[i][j]

            return hue

        # Merge channels into picture and return image
        hsi = cv2.merge((calc_hue(red, blue, green), calc_saturation(red, blue, green), calc_intensity(red, blue, green)))
        return hsi

def extract_features(image):
    # if zero dont input
    if np.any(image != [0, 0, 0]):
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Normalize RGB values to [0, 1] for accurate HSV conversion
        rgb_normalized = rgb.astype(np.float32) / 255.0

        # Convert RGB to HSV
        hsv = cv2.cvtColor(rgb_normalized, cv2.COLOR_RGB2HSV)

        # Convert to grayscale for GLCM
        gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)

        # Convert RGB to HSI
        hsi = rgb_to_hsi(image)

        # RGB
        r, g, b = np.mean(rgb[:, :, 0]), np.mean(rgb[:, :, 1]), np.mean(rgb[:, :, 2])

        # HSV
        h_hsv, s_hsv, v_hsv = np.mean(hsv[:, :, 0]), np.mean(hsv[:, :, 1]), np.mean(hsv[:, :, 2])

        # HSI (HLS approximation)
        h_hsi, s_hsi, i_hsi = np.mean(hsi[:, :, 0]), np.mean(hsi[:, :, 1]), np.mean(hsi[:, :, 2])

        # GLCM
        glcm = graycomatrix(gray, distances=[1], angles=[0, np.pi / 4, np.pi / 2, 3 * np.pi / 4],
                            symmetric=True, normed=True)
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        correlation = graycoprops(glcm, 'correlation')[0, 0]
        energy = graycoprops(glcm, 'energy')[0, 0]
        homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]

        return r, g, b, h_hsv, s_hsv, v_hsv, h_hsi, s_hsi, i_hsi, contrast, correlation, energy, homogeneity
    else:
        return None, None, None, None

# Load the trained K-NN model and scaler
knn_model = joblib.load('knn_model.pkl')
scaler = joblib.load('scaler.pkl')

# Create background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Function to classify an image
def classify_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(image)
    fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (31, 31), 0)

    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the contour with the largest area
        main_contour = max(contours, key=cv2.contourArea)

        # Draw bounding box around the main contour
        x, y, w, h = cv2.boundingRect(main_contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Zoom in to a specific area inside the bounding box
        zoom_factor = 3  # Adjust the zoom factor as needed
        center_x = x + w // 2
        center_y = y + h // 2
        zoom_size = min(w, h) // zoom_factor
        x_zoom = max(0, center_x - zoom_size // 2)
        y_zoom = max(0, center_y - zoom_size // 2)
        zoomed_image = image[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()

        # Resize the zoomed image to 100x100 pixels
        zoomed_resized = cv2.resize(zoomed_image, (32, 32))

        # Extract features from the zoomed-in frame with foreground mask
        zoomed_fg_mask = fg_mask[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()
        features = extract_features(cv2.bitwise_and(zoomed_image, zoomed_fg_mask))

    # Extract features
    features = extract_features(zoomed_resized)

    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])

        # Predict the grade
        grade = knn_model.predict(scaled_features)

        image = cv2.resize(image, (700, 700))
        # Draw the outline of the text on the zoomed image
        outline_thickness = 2
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)

        # Draw the text on the zoomed image
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the frame with bounding box
        cv2.imshow('Frame with Bounding Box', image)

        zoomed_resized = cv2.resize(zoomed_resized, (700, 700))
        # Draw the outline of the text on the zoomed image
        outline_thickness = 2
        cv2.putText(zoomed_resized, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)

        # Draw the text on the zoomed image
        cv2.putText(zoomed_resized, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the zoomed image with predicted grade
        cv2.imshow('Zoomed In with Predicted Grade', zoomed_resized)

        # Wait for a key press to close the windows
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        return grade[0]
    else:
        return "Error: Unable to extract features from the image."

# Function to classify an image and save the classified image
def classify_and_save_image(image_path, output_folder):
    start_time = time.time()
    process = psutil.Process(os.getpid())
    start_cpu = process.cpu_percent(interval=None)
    start_memory = process.memory_info().rss

    # Load the image
    image = cv2.imread(image_path)

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(image)
    fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (31, 31), 0)

    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the contour with the largest area
        main_contour = max(contours, key=cv2.contourArea)

        # Draw bounding box around the main contour
        x, y, w, h = cv2.boundingRect(main_contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Zoom in to a specific area inside the bounding box
        zoom_factor = 3  # Adjust the zoom factor as needed
        center_x = x + w // 2
        center_y = y + h // 2
        zoom_size = min(w, h) // zoom_factor
        x_zoom = max(0, center_x - zoom_size // 2)
        y_zoom = max(0, center_y - zoom_size // 2)
        zoomed_image = image[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()

        # Resize the zoomed image to 100x100 pixels
        zoomed_resized = cv2.resize(zoomed_image, (32, 32))

        # Extract features from the zoomed-in frame with foreground mask
        zoomed_fg_mask = fg_mask[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()
        features = extract_features(cv2.bitwise_and(zoomed_image, zoomed_fg_mask))

    # Extract features
    features = extract_features(zoomed_resized)

    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])

        # Predict the grade
        grade = knn_model.predict(scaled_features)

        image = cv2.resize(image, (800, 800))
        # Draw the predicted grade on the image
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Save the classified image to the output folder
        filename = os.path.basename(image_path)
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, image)

        end_time = time.time()
        end_cpu = process.cpu_percent(interval=None)
        end_memory = process.memory_info().rss

        processing_time = end_time - start_time
        cpu_usage = (end_cpu - start_cpu) / psutil.cpu_count()  # Convert to percent
        ram_usage = (end_memory - start_memory) / (1024 * 1024)  # Convert to MB

        return grade[0], processing_time, cpu_usage, ram_usage
    else:
        return "Error: Unable to extract features from the image.", 0, 0, 0

# Function to classify images in a folder and save the classified images to another folder
def classify_images_in_folder(input_folder, output_folder):
    processing_times = []
    cpu_usages = []
    ram_usages = []

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Construct the full path of the input image
            image_path = os.path.join(input_folder, filename)

            # Classify the image and save the classified image to the output folder
            predicted_grade, processing_time, cpu_usage, ram_usage = classify_and_save_image(image_path, output_folder)
            processing_times.append(processing_time)
            cpu_usages.append(cpu_usage)
            ram_usages.append(ram_usage)
            print("Image:", filename, "- Predicted Grade:", predicted_grade)
    
    print("Processing Time - Min:", np.min(processing_times), "Max:", np.max(processing_times), "Mean:", np.mean(processing_times))
    print("CPU Usage (%) - Min:", np.min(cpu_usages), "Max:", np.max(cpu_usages), "Mean:", np.mean(cpu_usages))
    print("RAM Usage (MB) - Min:", np.min(ram_usages), "Max:", np.max(ram_usages), "Mean:", np.mean(ram_usages))

# Call the function to classify images in the input folder and save the classified images to the output folder
input_folder = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\TUNA GRADE C\validation"
output_folder = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\TUNA GRADE C\val output2"
classify_images_in_folder(input_folder, output_folder)
print("B done")


Image: IMG20220515183513.jpg - Predicted Grade: C
Image: IMG20220515183514.jpg - Predicted Grade: C
Image: IMG20220515183515.jpg - Predicted Grade: C
Image: IMG20220515183517.jpg - Predicted Grade: C
Image: IMG20220515183807.jpg - Predicted Grade: C
Image: IMG20220515183809.jpg - Predicted Grade: C
Image: IMG20220515183811.jpg - Predicted Grade: C
Image: IMG20220515183812.jpg - Predicted Grade: C
Image: IMG20220515183813.jpg - Predicted Grade: C
Image: IMG20220515183815.jpg - Predicted Grade: C
Image: IMG20220515183817.jpg - Predicted Grade: C
Image: IMG20220515183819.jpg - Predicted Grade: C
Image: IMG20220515183820.jpg - Predicted Grade: C
Image: IMG20220515183821.jpg - Predicted Grade: C
Image: IMG20220515183822.jpg - Predicted Grade: C
Image: IMG20220515183823.jpg - Predicted Grade: C
Image: IMG20220515183825.jpg - Predicted Grade: C
Image: IMG20220515183826.jpg - Predicted Grade: C
Image: IMG20220516121850.jpg - Predicted Grade: B
Image: IMG20220516121851.jpg - Predicted Grade: C
